In [11]:
import pandas as pd
import numpy as np
import os

df = pd.read_csv('../data/mutation_matrix_cleaned.csv')

# save the first column
sample_id = df.iloc[:, 0]
sample_id

0        P-0000015-T01-IM3
1        P-0000041-T01-IM3
2        P-0000058-T01-IM3
3        P-0000098-T01-IM3
4        P-0000102-T02-IM3
               ...        
16904    P-0089479-T01-IM7
16905    P-0089690-T01-IM7
16906    P-0089687-T01-IM7
16907    P-0089672-T01-IM7
16908    P-0009406-T01-IM3
Name: Sample_ID, Length: 16909, dtype: object

In [12]:
# only columns from 1 to the end
df = df.iloc[:, 1:-1]

# remove the columns with less than 10% of non-zero values
df = df.loc[:, (df != 0).sum(axis=0) > (0.001 * df.shape[0])]
df

,ABL1,ABRAXAS1,ACVR1,AGO2,AKT1,AKT2,AKT3,ALB,ALK,ALOX12B,...,WWTR1,XIAP,XPO1,XRCC2,YAP1,YES1,ZFHX3,ZNRF3,ZRSR2,MSI
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16904,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
16905,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.0
16906,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
16907,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [13]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

# Keep track of missing data mask
data_np = df.to_numpy()
missing_mask = np.isnan(data_np)

#  Use Gene Mutation Probabilities for Initial Imputation
# gene_mutation_rates = np.nanmean(train_np, axis=0)
# train_np_filled = np.where(np.isnan(data_np), gene_mutation_rates, data_np)

imputer = KNNImputer(n_neighbors=5)
filled_np = imputer.fit_transform(data_np)
# Convert back to DataFrame
df_filled = pd.DataFrame(filled_np, columns=df.columns)
df_filled

,ABL1,ABRAXAS1,ACVR1,AGO2,AKT1,AKT2,AKT3,ALB,ALK,ALOX12B,...,WWTR1,XIAP,XPO1,XRCC2,YAP1,YES1,ZFHX3,ZNRF3,ZRSR2,MSI
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
16906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Define VAE Model
class BernoulliVAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(BernoulliVAE, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(64, latent_dim)
        self.fc_logvar = nn.Linear(64, latent_dim)
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()  # Ensures outputs are in [0,1]
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)  # Outputs probabilities in [0,1]

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
# Initialize VAE Model
# input_dim = train_np_filled.shape[1]  # Number of features
input_dim = df_filled.shape[1]  # Number of features
print(f"Input Dimension: {input_dim}")
latent_dim = 128
vae = BernoulliVAE(input_dim=input_dim, latent_dim=latent_dim)

# Define Optimizer and Loss Function
optimizer = optim.Adam(vae.parameters(), lr=0.001)
loss_fn = nn.BCELoss()  # Binary Cross-Entropy for binary data

# KL Divergence Loss Function
def kl_divergence(mu, logvar):
    return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())


Input Dimension: 496


In [15]:
# Training the VAE
num_epochs = 1000
batch_size = 64
filled_tensor = torch.tensor(filled_np, dtype=torch.float32)  # Convert filled data to tensor
mask = ~torch.isnan(filled_tensor)  # Create mask for observed values

for epoch in range(num_epochs):
    vae.train()
    optimizer.zero_grad()
    
    # Forward pass
    recon, mu, logvar = vae(filled_tensor)
    

    # Reconstruction loss (only for observed values)
    recon_loss = loss_fn(recon[mask], filled_tensor[mask])
    
    # KL Divergence loss
    kl_loss = kl_divergence(mu, logvar)
    
    # Total loss
    loss = recon_loss + 0.1 * kl_loss  # Adjust weighting if needed
    
    # Backpropagation
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 1046.4265
Epoch 50, Loss: 7.1687
Epoch 100, Loss: 1.8824
Epoch 150, Loss: 1.0434
Epoch 200, Loss: 0.6754
Epoch 250, Loss: 0.4800
Epoch 300, Loss: 0.3639
Epoch 350, Loss: 0.2876
Epoch 400, Loss: 0.2352
Epoch 450, Loss: 0.1976
Epoch 500, Loss: 0.1702
Epoch 550, Loss: 0.1497
Epoch 600, Loss: 0.1340
Epoch 650, Loss: 0.1218
Epoch 700, Loss: 0.1121
Epoch 750, Loss: 0.1048
Epoch 800, Loss: 0.0987
Epoch 850, Loss: 0.0939
Epoch 900, Loss: 0.0901
Epoch 950, Loss: 0.0868


In [16]:
# save the model
model_path = '../models/vae_model.pth'
torch.save(vae.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to ../models/vae_model.pth


In [ ]:
# Ensure VAE is in evaluation mode
vae.eval()

# Convert data to tensor
filled_tensor = torch.tensor(filled_np, dtype=torch.float32)

# Mask for missing values
mask = torch.isnan(filled_tensor)  # True where missing values exist

# Get VAE reconstruction
with torch.no_grad():
    recon, _, _ = vae(filled_tensor)

# Fill missing values with VAE predictions
filled_tensor[mask] = recon[mask]

# Convert continuous predictions to binary (Threshold at 0.5)
filled_tensor = (filled_tensor > 0.5).float()

# Convert back to DataFrame, index off
imputed_df = pd.DataFrame(filled_tensor.numpy(), columns=df.columns, index=df.index)

# Save to CSV
imputed_df.to_csv("imputed_mutation_matrix.csv")

print("Imputed mutation matrix saved successfully.")

Imputed mutation matrix saved successfully.
